# Neural Networks on MNIST Dataset

Import dependencies.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix


Read MNIST handwritten digits data from Pandas. Note the data is somewhat large so it is stored as a zipped CSV. 

In [ ]:
df = pd.read_csv('https://bit.ly/3ilJc2C', compression='zip', delimiter=",")
df

Separate the input and output variables. 

In [ ]:
X = df.values[:, :-1] / 255.0
Y = df.values[:, -1]


Print out the number of instances of each class. Stratify so that each class is sampled equally. 

In [ ]:

# Get a count of each group to ensure samples are equitably balanced
print(df.groupby(["class"]).agg({"class" : [np.size]}))

# Separate training and testing data
# Note that I use the 'stratify' parameter to ensure
# each class is proportionally represented in both sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
    test_size=.33, random_state=10, stratify=Y)


Train a neural network using the logistic function as the hidden layer activation function with 100 nodes. Set a higher learning rate of .01. 

In [ ]:

nn = MLPClassifier(solver='sgd',
                   hidden_layer_sizes=(100, ),
                   activation='logistic',
                   max_iter=480,
                   learning_rate_init=.1)

nn.fit(X_train, Y_train)

print("Test set score: %f" % nn.score(X_test, Y_test))

cf = confusion_matrix(y_true=Y_test, y_pred=nn.predict(X_test))
print(cf)

Display heat map for each digit character weights. 

In [ ]:
# Display heat map
import matplotlib.pyplot as plt
fig, axes = plt.subplots(4, 4)

# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = nn.coefs_[0].min(), nn.coefs_[0].max()
for coef, ax in zip(nn.coefs_[0].T, axes.ravel()):
    ax.matshow(coef.reshape(28, 28), cmap=plt.cm.gray, vmin=.5 * vmin, vmax=.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())